En esta sección elegiremos los mejores 4 modelos del notebook `modelosML.ipynb` para entrenarlos con Cross Validation. Para ello debemos de unir los datos de train con los de validacion, para que le modelo tenga más datos para entrenar y así poder generalizar mucho mejor. Los mejores modelos del notebook anterior son:
*   **LGBM**
*   **Random Forest**
*   **GBC**
*   **SVM**
             

A continuación leeremos los datos de train y validacion para concatenarlos

In [ ]:
import os
import pandas as pd
import numpy as np

# Cambiar el directorio de trabajo
ruta_nueva = 'C:/Users/ruben.morillas/Desktop/tfg/'  # Reemplaza 'TuUsuario' con tu nombre de usuario de Windows
os.chdir(ruta_nueva)

# Mostrar el directorio actual para confirmar el cambio
directorio_actual = os.getcwd()
print("Directorio actual:", directorio_actual)

In [ ]:
import pandas as pd
path_actual = os.getcwd()
subdirectorio = 'datas'
file_train = 'df_train.csv'
path_train = os.path.join(path_actual, subdirectorio, file_train)
df_train = pd.read_csv(path_train, index_col=None)

In [ ]:
file_valid = 'df_valid.csv'
path_valid = os.path.join(path_actual, subdirectorio, file_valid)
df_valid = pd.read_csv(path_valid, index_col=None)

In [ ]:
df_train.drop(['trueE'], axis=1, inplace=True)
df_valid.drop(['trueE'], axis=1, inplace=True)

In [ ]:
df_combinado = pd.concat([df_train, df_valid])

In [ ]:
def filtrado_datos(df, N):
    kaones = []
    labels = []

    df_sorted = df.sort_values(by=['eventID', 'hitTime'], ascending=[True, False])

    for eventID, grupo in df_sorted.groupby('eventID'):
        pdgCodes = grupo['PDGcode'].unique()

        for pdgCode in pdgCodes:
            grupo_filtrado = grupo[grupo['PDGcode'] == pdgCode]
            grupo_ordenado = grupo_filtrado.head(N)

            # Inicializar arrays para el padding
            hitX_padded = np.zeros(N)
            hitY_padded = np.zeros(N)
            hitZ_padded = np.zeros(N)
            hitInteg_padded = np.zeros(N)

            # Separar y aplicar padding a los valores de hitX, hitY, hitZ, hitInteg
            hitX_padded[:len(grupo_ordenado['hitX'])] = grupo_ordenado['hitX']
            hitY_padded[:len(grupo_ordenado['hitY'])] = grupo_ordenado['hitY']
            hitZ_padded[:len(grupo_ordenado['hitZ'])] = grupo_ordenado['hitZ']
            hitInteg_padded[:len(grupo_ordenado['hitInteg'])] = grupo_ordenado['hitInteg']

            # Concatenar los valores ya con el padding aplicado
            hit_values_reorganized = np.concatenate([hitX_padded, hitY_padded, hitZ_padded, hitInteg_padded])

            kaones.append(hit_values_reorganized)

            # Modificar las etiquetas de 211 a 0 y de 321 a 1
            if pdgCode == 211:
                labels.append(0)
            elif pdgCode == 321:
                labels.append(1)

    return np.array(kaones), np.array(labels)

In [ ]:
X, y = filtrado_datos(df_combinado, N=400)

Importaremos las bibliotecas de los distintos modelos y también el de la CV, para poder entrenar.

In [ ]:
import xgboost as xgb
from lightgbm import LGBMClassifier # LightGBM
from sklearn.svm import SVC # SVM
from sklearn.ensemble import RandomForestClassifier # Random Forest
from sklearn.model_selection import cross_val_score

In [ ]:
RF = RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=None, max_features='sqrt', random_state=42)
scores = cross_val_score(RF, X, y, cv=5, scoring='accuracy')
scores
